In [8]:
import librosa
import numpy as np
import torch
import torch.nn as nn

In [3]:
class CRNN(nn.Module):
    def __init__(self):
        super(CRNN, self).__init__()
        self.CNNLayer = nn.Sequential(
            nn.Conv2d(1, 64, (2,2), stride=(2,2)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)),
            
            nn.Conv2d(64, 64, (2,2), stride=(2,2)),
            nn.ReLU(),
            
            nn.Conv2d(64, 64, (2,2), stride=(2,2)),
            nn.ReLU(),
        )
        
        self.flatten = nn.Flatten()
        
        self.LinearLayer = nn.Sequential(
            nn.Linear(12288, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256, 4),
            nn.Softmax(dim=1),
        )
    
    def forward(self, x):
        conv_output = self.CNNLayer(x)
        flatten = self.flatten(conv_output)
        linear_output = self.LinearLayer(flatten)
        
        return linear_output

In [6]:
model = CRNN()
model = torch.load('./4direction_best.pt').to('cuda')
model.eval()

audio, sr = librosa.load('./옆2.m4a', sr=16000, mono=False)
audio = audio[:, :16000]

win_length = 320 # 320은 20ms 의미

/tmp/ipykernel_25445/2405761665.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load('./옆2.m4a', sr=16000, mono=False)
/home/20223164/.conda/envs/ssl/lib/python3.11/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [13]:
import time

def inference(audio):
    with torch.inference_mode():
        stft1 = librosa.stft(audio[0], n_fft=512, hop_length=win_length, win_length=win_length)
        stft2 = librosa.stft(audio[1], n_fft=512, hop_length=win_length, win_length=win_length)

        feature_set1 = torch.tensor(np.concatenate((stft1.real, stft2.real), axis=0))
        feature_set2 = torch.tensor(np.concatenate((stft1.imag, stft2.imag), axis=0))

        feature = torch.concatenate((feature_set1, feature_set2), axis=0)

        feature = feature.unsqueeze(0).unsqueeze(0).to('cuda')
        output = model(feature).argmax(dim=1)
    
    return output

s_time = time.time()
output = inference(audio)
print(time.time()-s_time)

0.028270721435546875


In [12]:
output

tensor([0], device='cuda:0')